## Reproducing baseline model from tensorflow tutorial
#### trained on old data: 76k of 3 channels images
#### conv1: 32 features, 5 x 5, max pool 2, 2
#### conv2: 64 features, 5 x 5, max pool 2, 2
#### fc1: 1024
#### no dropout

In [ ]:
import gen_input
import numpy as np
import os
from datetime import datetime
from time import time
import tensorflow as tf

In [ ]:
train_test_valid_split = [1., 0., 0.]
svhn_train = gen_input.read_data_sets("data/train_32x32.mat", train_test_valid_split).train
svhn_test = gen_input.read_data_sets("data/test_32x32.mat", train_test_valid_split).train

print svhn_train.images[0].shape
print svhn_train.images.shape
print svhn_test.images[0].shape
print svhn_test.images.shape

## Model Wrappers

In [ ]:
def weight_variable(shape, stddev=1.0):
  initial = tf.truncated_normal(shape, stddev)
  return tf.Variable(initial)

def bias_variable(shape, default_bias=0.0):
  initial = tf.constant(default_bias, shape=shape)
  return tf.Variable(initial)

def convlayer(layer_name, input_tensor, receptive_field, channels_in, channels_out,
              padding='SAME', stride=1, act=tf.nn.relu, decay=0,
              pool=True, pooler=tf.nn.max_pool, pool_size=2, pool_stride=2, pool_padding='SAME',
              batch_norm=False, training=True):
  
  with tf.name_scope(layer_name):
    with tf.name_scope('weights'):
      weights = weight_variable([receptive_field, receptive_field, channels_in, channels_out])
      
      if decay > 0:
          weight_decay = tf.multiply(tf.nn.l2_loss(weights), decay, name='weight_decay')
          tf.add_to_collection('losses', weight_decay)
  
    with tf.name_scope('biases'):
      biases = bias_variable([channels_out])
      
    with tf.name_scope('W_conv_x_plus_b'):
      preactivate = tf.nn.conv2d(input_tensor, 
                    weights, strides=[1, stride, stride, 1], padding=padding) + biases
    
    if batch_norm:
      with tf.name_scope('batchnorm'):
        normed = tf.layers.batch_normalization(preactivate, training=training)
      activations = act(normed, name='activation')
    else:
      activations = act(preactivate, name='activation')  
    
    if pool:
      max_pool = pooler(activations, ksize=[1, pool_size, pool_size, 1], 
                      strides=[1, pool_stride, pool_stride, 1],
                      padding=pool_padding)
      return max_pool
    else: 
      return activations
    
def nn_layer(layer_name, input_tensor, input_dim, output_dim, act=tf.nn.relu, decay=0):
  with tf.name_scope(layer_name):
    with tf.name_scope('weights'):
      weights = weight_variable([input_dim, output_dim])
      
      if decay > 0:
        weight_decay = tf.multiply(tf.nn.l2_loss(weights), decay, name='weight_decay')
        tf.add_to_collection('losses', weight_decay)

    with tf.name_scope('biases'):
      biases = bias_variable([output_dim])
    with tf.name_scope('Wx_plus_b'):
      preactivate = tf.matmul(input_tensor, weights) + biases
    activations = act(preactivate, name='activation')
    return activations
  
def flat_dimension(tensor):
  dim = 1 # Compute how many numbers we have, ignoring the batch size
  for d in tensor.get_shape()[1:].as_list():
    dim *= d
  return dim

# Normalize by subtracting per image, per channel means
def normalize_batch(batch):
  per_img_ch_means = batch.mean(axis=1)
  return batch - per_img_ch_means[:, np.newaxis, :]

In [ ]:
def run():
  # RESET TF GRAPH, just in case
  tf.reset_default_graph()
  
  ### Place holders ###
  with tf.name_scope('test_train_variables'):
    batch_norm_train_mode = tf.placeholder(tf.bool) # for batch_norm mode
    keep_prob = tf.placeholder(tf.float32) # for drop out
    
    # Optionally track that place holders are correctly set at test and train tme
    tf.summary.scalar('training', tf.to_int32(batch_norm_train_mode, name='ToInt32'))
    tf.summary.scalar('dropout_keep_probability', keep_prob)
  
  with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, 32*32, 3], name="x-input") 
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y-input")
    
  ###################
  ##### Network #####
  ###################
  with tf.name_scope('input_reshape'):
    input_reshaped = tf.reshape(x, [-1, 32, 32, 3])
#     tf.summary.image('input', input_reshaped, 5) # Optionally save 5 images to ensure reshape is working

  conv1 = convlayer(layer_name='conv1', input_tensor=input_reshaped, receptive_field=5, 
                      channels_in=3, channels_out=32, pool=True, pool_size=2, pool_stride=2,
                      batch_norm=False, training=batch_norm_train_mode)
  
  conv2 = convlayer(layer_name='conv2', input_tensor=conv1, receptive_field=5, 
                    channels_in=32, channels_out=64, pool=True, pool_size=2, pool_stride=2,
                    batch_norm=False, training=batch_norm_train_mode)
  
  last_conv = conv2
  with tf.name_scope('conv2_flatten'):
    conv_reshaped = tf.reshape(last_conv, [-1, flat_dimension(last_conv)])

  fc1 = nn_layer(layer_name='fc1', input_tensor=conv_reshaped, input_dim=flat_dimension(last_conv), output_dim=1024, decay=fc_decay)
  dropped1 = tf.nn.dropout(fc1, keep_prob)

  # Do not apply softmax activation yet! use the identity
  y = nn_layer(layer_name='output', input_tensor=dropped1, input_dim=1024, output_dim=10, act=tf.identity) 

  ### Losses and Accuracy ###
  # Cross-Entropy Loss
  with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
    with tf.name_scope('total'):
      cross_entropy = tf.reduce_mean(diff)
      tf.add_to_collection('losses', cross_entropy)
  tf.summary.scalar('cross_entropy', cross_entropy)
  # Total loss (weight decay + cross-entropy)
  total_loss = tf.add_n(tf.get_collection('losses'), name='total_loss')

  with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

  # Other metrics
  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('accuracy', accuracy)

  # Might be needed for batch norm
  extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)  
  # Initializing the variables
  init = tf.global_variables_initializer()

  merged_summaries = tf.summary.merge_all()
  
  # for saving the model in the end
  saver = tf.train.Saver()

  ts = datetime.now().strftime('%Y%m%d_%H%M')
  logs_path = "logs/{}/".format(ts)
  print "-"* 70
  pwd = os.getcwd()+"/"
  print("Run the following to start tensorboard server:\n" \
        "tensorboard --logdir=/{}{}".format(pwd, logs_path))

  ##########################################
  ##                                      ##
  ##           Launch the graph           ##
  ##                                      ##
  ##########################################
  
  # Fill in the place holders depending on the context (training? testing?)
  def feed_dict(mode):
    if mode == 'train':
      batch_x, batch_y = svhn_train.next_batch(batch_size)
      keep_proba = train_keep_prob
      training_mode = True

    elif mode == 'train_no_dropout':
      batch_x, batch_y = svhn_train.next_batch(test_batch_size)
      keep_proba = 1.0
      training_mode = False

    elif mode == 'test_no_dropout':
      batch_x, batch_y = svhn_test.next_batch(test_batch_size)
      keep_proba = 1.0
      training_mode = False

    batch_x = normalize_batch(batch_x) # Subtract per image mean
    return {x: batch_x, y_: batch_y, keep_prob: keep_proba, batch_norm_train_mode: training_mode} # can't name values same as keys

  with tf.Session() as sess:
    t_start = time()
    sess.run(init)
    train_writer = tf.summary.FileWriter(logs_path + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(logs_path + '/test')
    # Training loop
    for epoch in xrange(training_epochs):
      for batch_num in xrange(total_batches):
        if batch_num % test_every == test_every - 1:

          # Record summaries and accuracy on the *test* set
          summary, acc = sess.run([merged_summaries, accuracy], feed_dict=feed_dict(mode='test_no_dropout'))
          test_writer.add_summary(summary, epoch * total_batches + batch_num)

          # To compare against *training* set (apples to apples comparison)
          summary, _, _ = sess.run([merged_summaries, optimizer, extra_update_ops], feed_dict=feed_dict(mode='train_no_dropout'))
          train_writer.add_summary(summary, epoch * total_batches + batch_num)

          # Print occasional progress
          print('Test accuracy at epoch %s: batch %s: %s' % (epoch, batch_num, acc))
        else:
          sess.run([optimizer, extra_update_ops], feed_dict=feed_dict(mode='train'))
    
    t_end = time()
    elapsed_mins = (t_end - t_start) / 60.0
    print "\nOptimization Finished! in {} minutes".format(elapsed_mins)
    
    # Save down the current model
    if not os.path.exists("models"): os.makedirs("models")
    saver.save(sess, "models/{}".format(model_name))

In [ ]:
##########################################
##                                      ##
##              Parameters              ##
##                                      ##
##########################################

# Training Parameters
learning_rate = 0.001
training_epochs = 3

batch_size = 40 # 100
test_batch_size = 40 # 5000
test_every = 100

total_batches = int(svhn_train.num_examples / batch_size)

# Regularization
fc_decay = 0
train_keep_prob = 1.0

model_name = "baseline_old_data"

run()

## Testing inference

In [ ]:
# saver = tf.train.Saver()
# x_batch, y_batch = svhn_test.next_batch(10)

# with tf.Session() as sess:
#   saver.restore(sess, "models/{}".format(model_name))
#   print "model restored"
#   #logits

#   logits = tf.get_collection("logits")[0]
#   x = tf.get_collection("x")[0]
#   y_ = tf.get_collection("y_")[0]
#   keep_prob = tf.get_collection("keep_prob")[0]
  
#   predictions = sess.run(logits, feed_dict={x: x_batch, y_: y_batch, keep_prob: 1.0})
#   y_preds = tf.argmax(predictions, 1)
#   y_actual = tf.argmax(y_batch, 1)
  
#   ## tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
#   print sess.run(y_preds)
#   print sess.run(y_actual)